In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [39]:
Box(12, 23, (3,4)).sample()

array([[21.781164, 18.203114, 15.047821, 16.386751],
       [19.865635, 21.99123 , 16.156376, 13.621446],
       [22.153965, 13.256039, 14.029147, 19.982533]], dtype=float32)

In [3]:
Env??

In [12]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        
    def step(self, action):
        
        self.state += action-1
        
        self.shower_length -= 1
        
        if self.state >=  37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        pass
    def reset(self):
        self.shower_length = 60
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        return self.state

In [13]:
env = ShowerEnv()

In [14]:
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [15]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:-16
Episode:2 Score:36
Episode:3 Score:-24
Episode:4 Score:-8
Episode:5 Score:-20


In [16]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy', env, verbose =1, tensorboard_log = log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -26.1    |
| time/              |          |
|    fps             | 1962     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -23.9       |
| time/                   |             |
|    fps                  | 1164        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010972126 |
|    clip_fraction        | 0.0598      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.08       |
|    explained_variance   | -0.00153    |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -22.7       |
| time/                   |             |
|    fps                  | 845         |
|    iterations           | 11          |
|    time_elapsed         | 26          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010485906 |
|    clip_fraction        | 0.0928      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.965      |
|    explained_variance   | -0.00298    |
|    learning_rate        | 0.0003      |
|    loss                 | 43.6        |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.0063     |
|    value_loss           | 82          |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

In [34]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(24.0, 54.99090833947008)

In [24]:
save_path = os.path.join('Training', 'Saved Models', 'PPO_custom_RL')
model.save(save_path)

In [25]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:-56
Episode:2 Score:36
Episode:3 Score:-38
Episode:4 Score:-8
Episode:5 Score:16
